# RAG를 이용하여 국방백서를 알려주는 간단한 챗봇 만들기

최근 자연어처리 분야에서 검색 증강 생성 (Retrieval-Augmented Generation, RAG)은 각광받는 기술입니다.

RAG는 사용자가 질문을 보내면, 사전에 만들어둔 데이터베이스에서 검색하여

검색한 내용을 사용자 입력에 붙여서 (Augment) 모델에게 함께 전달합니다. 


이하 예제 코드는 VectorDB 라이브러리 ChromaDB와 LLM 고수준 프레임워크 Ollama와 Langchain를 이용하여,
- PDF 파일로 부터 텍스트를 추출하고 이를 쪼개어서 문서 집합을 생성해봅니다.
- ~~생성한 문서집합으로부터 VectorDB를 구축한다.~~
  - 미리 생성된 VectorDB를 사용합니다.
- LLM과 VectorDB 검색모듈을 연결하여 국방백서에 특화된 챗봇을 제작해봅니다.

# Requirements

torch는 이미 다운로드 받았다고 가정합니다. (colab은 이미 설치되어 있습니다.)

**(colab의 경우) 앞으로 진행하기 전에 런타임 유형을 변경하시는 것을 추천드립니다!**

![image.png](https://github.com/xaiseung/NLP_Policy_test/blob/main/images/colab_session.png?raw=true)

바꾸지 않아도 실행은 되지만, (CPU) 8분 vs (GPU) 24초로 크게 차이가 나게 됩니다.


ollama 설치
- 빠른 LLM 추론을 위한 프레임워크입니다.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

Python 패키지
- pymupdf
- ollama
- chromadb
- huggingface-hub
- sentence-transformers
- transformers
- langchain
- langchain-community
- langchain-ollama


In [ ]:
!pip install pymupdf ollama chromadb huggingface-hub sentence-transformers transformers langchain langchain-community langchain-ollama gdown

# 학습자료 다운로드

In [ ]:
!gdown https://drive.google.com/uc?id=1WN1efc4LjCX4GO4uRtiRGAhTIC6fH7mk
!tar -xzvf "[06]materials.tar.gz"

# Ollama에 모델 등록하기

Ollama는 대규모 언어 모델 (LLM) 추론을 로컬에서 빠르게 실행할 수 있게 해주는 도구입니다.

제공한 학습자료에서 `./models/bllossom3b` 에는 우리가 사용할 자연어 모델 가중치(파라미터)들이 있습니다. 

Ollama에서 사용하기 위해서는 해당 모델을 등록해야 합니다.

```
ollama create bllossom3b -f ./models/bllossom3b/Modelfile
```

In [ ]:
# ollama 시작
!nohup ollama serve &
# bllossom3b 모델 등록
!ollama create bllossom3b -f ./models/bllossom3b/Modelfile

ollama에 등록한 모델을 Langchain 프레임워크에 연결하여, 자유롭게 대화해볼 수 있습니다.

In [ ]:
import ollama
from langchain_ollama.chat_models import ChatOllama

# Ollama 를 이용해 로컬에서 LLM 실행
## llama3-ko-instruct 모델 다운로드는 Ollama 사용법 참조

model_id = "bllossom3b"
model = ChatOllama(model=model_id, temperature=0)

In [ ]:
print(model.invoke("안녕하세요?").content)

`bllossom3b`는 한국어에 튜닝된 공개 대규모 언어모델이며, 뒤의 `3b` (3 Billions)는 모델의 크기, 즉 가중치의 개수를 의미합니다.

최근 모델 기조에 비하면 모델크기가 작은편이고, 특정 도메인에 한정된 추가 학습이 이루어지지 않았기 때문에 큰 성능을 기대하기는 어렵지만,

예제를 시연하는데에는 적합한 크기입니다.

# PDF 데이터 (국방백서) 전처리하기

이제 검색 증강 생성 (RAG)를 위한 데이터베이스를 구축해봅시다.

먼저, 검색에 사용할 말뭉치가 필요합니다. 우리가 해결하고자 하는 문제에서 도움이 되는 지식을 포함한 텍스트들이여야 합니다.

말뭉치는 .txt 파일부터, .pdf, .docx, .html 등등을 사용할 수 있습니다.

해당 예제에서는 2022년 국방백서 pdf 파일을 이용하여 RAG를 위한 데이터베이스를 구축해볼 것입니다.

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDFLoader 을 이용해 PDF 파일 로드
loader = PyMuPDFLoader("2022 국방백서.pdf")
pages = loader.load()

# 전처리
for page in pages:
    page.page_content = page.page_content.replace("\n", " ").replace("  ", " ").replace(". ", ".\n") 

`pages`변수는 각 페이지를 나누어 저장해둔 리스트입니다.

테스트로 `pages[2]`를 출력해봅시다.

In [ ]:
print(pages[2].page_content)

다음은 각 페이지를 짧은 문장으로 쪼개는 과정입니다.

짧은 문장으로 나누어지고 나면, 이것이 DB에 각각 저장되는 단위, 문서가 됩니다.

- `chunk_size`: 자르는 문서의 크기입니다.
- `chunk_overlap`: 문서를 자를 때 인접한 두 문서가 해당 개수만큼 겹치게 자릅니다.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 페이지의 텍스트들을 더 작은 단위로 자르기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=32,
)
docs = text_splitter.split_documents(pages)

In [ ]:
# 문서의 개수
len(docs)

테스트로 `docs[0]`을 출력해봅시다.

In [ ]:
print(docs[0])

# 벡터 저장소 생성

벡터 저장소 (Vector Store, VectorDB)란 텍스트, 이미지, 음성, 센서데이터 등 다양한 종류의 데이터들을 벡터로 변환하여 색인 (indexing)한 뒤

색인된 벡터를 바탕으로 빠르게 검색할 수 있게 해주는 모듈입니다.

우리가 만들 것은 국방백서에 대한 질문이 입력으로 들어왔을 때, 질문에 관련된 정보를 빠르게 찾아내는 벡터저장소를 만들고자 하는 것입니다.

이하 코드는 `bge-m3` 모델을 이용하여 문서를 벡터화하고 ChromaDB를 이용하여 벡터 저장소를 만드는 예제입니다.

(실행하면 오래 걸리므로, 미리 만들어진 결과물을 사용할 것입니다.)

In [ ]:
"""
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import os

# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':device},
    encode_kwargs={'normalize_embeddings':True},
)

# 벡터 저장소 경로 설정
## 현재 경로에 'vectorstore' 경로 생성
vectorstore_path = 'vectorstore'
os.makedirs(vectorstore_path, exist_ok=True)

# 벡터 저장소 생성 및 저장
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=vectorstore_path)
#vectorstore = Chroma(embedding_function=embeddings, persist_directory=vectorstore_path)

# 벡터스토어 데이터를 디스크에 저장
vectorstore.persist()
print("Vectorstore created and persisted")
"""

## 만들어진 벡터 저장소 불러오기

- 처음 실행시에는 문서 임베딩 모델을 다운로드 받는 시간이 필요합니다.

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# 저장된 VectorStore 로드
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':device},
    encode_kwargs={'normalize_embeddings':True},
)
vectorstore_path = 'vectorstore'
vectorstore = Chroma(embedding_function=embeddings, persist_directory=vectorstore_path)



# 국방백서 챗봇 파이프라인 구축하기

## 언어모델 불러오기 + 검색 모듈 초기화

Ollama로 등록한 `bllossom3b` 모델을 불러오고, 벡터 저장소를 검색모듈로 변환합니다.

In [ ]:
from langchain_community.chat_models import ChatOllama

# Ollama 를 이용해 로컬에서 LLM 실행
## llama3-ko-instruct 모델 다운로드는 Ollama 사용법 참조

model = ChatOllama(model="bllossom3b", temperature=0)
retriever = vectorstore.as_retriever(search_kwargs={'k': 4})

## 챗봇 파이프라인 구축

- 대화용 프롬프트를 생성합니다.
  - 중괄호롤 묶인 키워드 (예: `{context}`)가 프롬프트의 빈칸(placeholder)가 되어, 입력을 키워드에 대응시켜 이 자리를 채우도록 할 수 있습니다.
- 프롬프트에 검색모듈의 검색 결과와 사용자의 입력을 대응시키도록 연결합니다.
- 이를 모델에 연결하고 표준 출력을 사용하도록 합니다.
- invoke() 메소드를 사용하여, 챗봇을 이용해봅니다.

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


# Prompt 템플릿 생성
template = '''친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하세요. 모든 대답은 한국어(Korean)으로 대답하세요.
다음 문맥(context)는 질문에 도움이 될 수도 있고 아닐 수도 있습니다. 문맥 내애서 북한의 내용인지 남한의 내용인지 잘 구분하세요. 문맥을 바탕으로 알 수 없다면 모르겠다고 답변하세요.

context:
{context}

Question: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# 검색된 문서 텍스트 전처리
def format_docs(docs):
    return '\n\n\n'.join([f"## Docs{i+1}\n{d.page_content}" for i, d in enumerate(docs)])

# 검색 체인 - 검색 모듈과 사용자 입력을 정해진 placeholder에 배정한다.
retrieve_chain = {'context': retriever | format_docs, 'question': RunnablePassthrough()} | prompt
# RAG Chain 연결
rag_chain = (
    retrieve_chain
    | model
    | StrOutputParser()
)

챗봇 파이프라인이 만들어졌습니다. 다음과 같이 질문을 줄 수 있습니다.

In [ ]:
# Chain 실행
query = "국제 사회의 안보 불확실성이 증대된 이유가 무엇입니까?"
print("Query:", query)
print("Answer:")

async for chunk in rag_chain.astream(query):
  print(chunk, end ="")
print()



<hr>


우리의 질문에 대해서 문서 검색이 어떻게 이루어졌고 모델이 받는 입력이 들어갔는지 보기위해

RAG 검색 체인만 실행해 확인해봅시다.

In [ ]:
print(retrieve_chain.invoke(query).to_string())



<hr>

또 다른 질문 예시입니다.

In [ ]:
query = "주한미군에 대해서 알려주세요."
print("Query:", query)
print("Answer:")

async for chunk in rag_chain.astream(query):
  print(chunk, end ="")
print()

In [ ]:
print(retrieve_chain.invoke(query).to_string())

---
모를 것 같은 질문으로 넣어보고 어떻게 대답하는지 확인해봅시다.

In [ ]:
query = "한산대첩에서 사용한 우리나라의 진법이 무엇인지 아시나요?"
print("Query:", query)
print("Answer:")

async for chunk in rag_chain.astream(query):
  print(chunk, end ="")
print()

In [ ]:
print(retrieve_chain.invoke(query).to_string())